In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from sklearn import preprocessing, neighbors, svm
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error
from scipy.stats import truncnorm
from scipy.optimize import curve_fit
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
from torch_geometric.nn import  global_add_pool

In [2]:
df = pd.read_csv('Feature_vector_full.csv')

In [3]:
X = df.drop(['target_energy'], axis=1)
y = df['target_energy']

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Step 1: Split into temp (train+val) and test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Split temp into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [ ]:

# class BasicGNN(nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(BasicGNN, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         # x: [num_nodes, in_channels]
#         # edge_index: [2, num_edges]
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.conv2(x, edge_index)
#         return x


In [6]:
class FCNEmbedder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(FCNEmbedder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, latent_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [7]:
def build_knn_graph(embeddings, k=6):
    embeddings = embeddings.detach().cpu().numpy()
    knn = NearestNeighbors(n_neighbors=k + 1)  # +1 to skip self-loop
    knn.fit(embeddings)
    distances, indices = knn.kneighbors(embeddings)

    # Build edge_index
    edge_index = []
    for i, neighbors in enumerate(indices):
        for j in neighbors[1:]:  # Skip self
            edge_index.append([i, j])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return edge_index


In [8]:
class GNNRegressor(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, 1)  # predict scalar per graph

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_add_pool(x, batch)  # Aggregate all node features per graph
        x = self.fc(x)
        return x


In [9]:
# Prepare tensors

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)

# Define models
fcn = FCNEmbedder(input_dim=X.shape[1], latent_dim=16)
# gnn = SimpleGNN(latent_dim=16, hidden_dim=32, output_dim=1)

# Get latent embeddings
with torch.no_grad():
    latent = fcn(X_train_tensor)

# Build graph
edge_index = build_knn_graph(latent, k=6)

# Create PyG data object
data = Data(x=latent, edge_index=edge_index, y=y_train_tensor.view(-1, 1))


In [10]:
# output = gnn(data.x, data.edge_index)

In [11]:
# output.shape

In [12]:
# Instantiate models
fcn = FCNEmbedder(input_dim=X_train.shape[1], latent_dim=16)
gnn = GNNRegressor(in_channels=16, hidden_channels=32)

# Optimizer and loss
optimizer = torch.optim.Adam(list(fcn.parameters()) + list(gnn.parameters()), lr=1e-3)
loss_fn = nn.MSELoss()


In [13]:
for epoch in range(100):
    fcn.train()
    gnn.train()

    # Get embeddings
    latent = fcn(X_train_tensor)

    # Build kNN graph
    edge_index = build_knn_graph(latent, k=6)

    # Forward through GNN
    batch = torch.zeros(latent.size(0), dtype=torch.long)  # All nodes belong to 1 graph

    y_pred = gnn(latent, edge_index, batch)

    # Compute loss
    loss = loss_fn(y_pred, y_train_tensor)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: Loss = 1060375166976.0000


/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect r

Epoch 10: Loss = 2331473152.0000


/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/bapi_cms/miniconda3/envs/temporary/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect r

KeyboardInterrupt: 